In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import imdb

Using TensorFlow backend.


In [3]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

In [4]:
print('訓練總筆數:', len(x_train))
print('測試總筆數:', len(x_test))

訓練總筆數: 25000
測試總筆數: 25000


In [5]:
 x_train[24999]

[1,
 17,
 6,
 194,
 337,
 7,
 4,
 204,
 22,
 45,
 254,
 8,
 106,
 14,
 123,
 4,
 2,
 270,
 2,
 5,
 2,
 2,
 732,
 2098,
 101,
 405,
 39,
 14,
 1034,
 4,
 1310,
 9,
 115,
 50,
 305,
 12,
 47,
 4,
 168,
 5,
 235,
 7,
 38,
 111,
 699,
 102,
 7,
 4,
 4039,
 9245,
 9,
 24,
 6,
 78,
 1099,
 17,
 2345,
 2,
 21,
 27,
 9685,
 6139,
 5,
 2,
 1603,
 92,
 1183,
 4,
 1310,
 7,
 4,
 204,
 42,
 97,
 90,
 35,
 221,
 109,
 29,
 127,
 27,
 118,
 8,
 97,
 12,
 157,
 21,
 6789,
 2,
 9,
 6,
 66,
 78,
 1099,
 4,
 631,
 1191,
 5,
 2642,
 272,
 191,
 1070,
 6,
 7585,
 8,
 2197,
 2,
 2,
 544,
 5,
 383,
 1271,
 848,
 1468,
 2,
 497,
 2,
 8,
 1597,
 8778,
 2,
 21,
 60,
 27,
 239,
 9,
 43,
 8368,
 209,
 405,
 10,
 10,
 12,
 764,
 40,
 4,
 248,
 20,
 12,
 16,
 5,
 174,
 1791,
 72,
 7,
 51,
 6,
 1739,
 22,
 4,
 204,
 131,
 9]

In [6]:
len(x_train[24999])

153

In [7]:
len(x_train[9982])

156

In [8]:
len(x_train[9487])

104

In [9]:
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [10]:
y_train[24999]

0

## 設計我們的情意分析RNN

- 1000>3
- RNN:10
- Dense(1)

In [17]:
from keras.preprocessing import sequence

In [18]:
x_train = sequence.pad_sequences(x_train, maxlen=150)
x_test = sequence.pad_sequences(x_test, maxlen=150)

In [19]:
x_train.shape

(25000, 150)

## 打造我們的RNN

### Step 1 建置RNN神經網路

In [39]:
N = 3 # 文字要壓到 N 維, 固定壓到3維
K = 50 # LSTM 有 K 個神經元

In [40]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

In [41]:
model = Sequential()

In [42]:
model.add(Embedding(10000,N))

In [43]:
# LSTM放入k個cells
model.add(LSTM(K))

In [44]:
#2類判斷的activation使用sigmoid
model.add(Dense(1, activation="sigmoid"))

### Step2 組裝

In [45]:
model.compile(loss = 'binary_crossentropy',
             optimizer='adam',
              metrics = ["accuracy"])

In [46]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 3)           30000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                10800     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 40,851
Trainable params: 40,851
Non-trainable params: 0
_________________________________________________________________


In [47]:
# 計算方式：有
((4*13)+4)*K

2800

In [48]:
model.fit(x_train, y_train,
         batch_size=32,
         epochs=5)

//anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
25000/25000 [==============================] - 46s 2ms/step - loss: 0.5083 - accuracy: 0.7250
Epoch 2/5
25000/25000 [==============================] - 46s 2ms/step - loss: 0.2744 - accuracy: 0.8914
Epoch 3/5
25000/25000 [==============================] - 47s 2ms/step - loss: 0.2223 - accuracy: 0.9160
Epoch 4/5
25000/25000 [==============================] - 47s 2ms/step - loss: 0.1880 - accuracy: 0.9305
Epoch 5/5
25000/25000 [==============================] - 47s 2ms/step - loss: 0.1617 - accuracy: 0.9416


In [49]:
score = model.evaluate(x_test, y_test)

25000/25000 [==============================] - 11s 422us/step


In [50]:
print(f'測試資料的 loss = {score[0]}')
print(f'測試資正確率 = {score[1]}')

測試資料的 loss = 0.40138965051651
測試資正確率 = 0.8536800146102905


### 把model存起來

In [51]:
model_json = model.to_json()
open('imdb_model_arch.json',
     'w').write(model_json)
model.save_weights('imdb_model_weights.h5')